In [2]:
import pycolmap
from pathlib import Path
import shutil
import urllib.request
import zipfile
import io
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import open3d as o3d
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [47]:
def plot_cameras(reconstrucao):

    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    for image_id, image in reconstruction.images.items():
        #Posição da câmera
        cam_fromworld = image.cam_from_world()
        camera_pos = cam_fromworld.translation
        ax.scatter(camera_pos[0], camera_pos[1], camera_pos[2], c='red', marker='o')

        # Calcula a orientação da câmera
        rotation_matrix = cam_fromworld.rotation.matrix()
        forward_dir = rotation_matrix @ np.array([0, 0, 1])  # Vetor forward da câmera
        ax.quiver(
            camera_pos[0], camera_pos[1], camera_pos[2],
            forward_dir[0], forward_dir[1], forward_dir[2],
            length=0.1, color='blue', arrow_length_ratio=0.1
        )

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('Camera Positions and Directions')
    plt.show()

In [80]:
def vis_estatica_camera_objeto(reconstruction):
    points_3d = []
    
    for point3D_id, point3D in reconstruction.points3D.items():
        points_3d.append(point3D.xyz)  

    points_3d = np.array(points_3d)  

    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    ax.scatter(
        points_3d[:,0],  # X
        points_3d[:,1],  # Y
        points_3d[:,2],  # Z
        s=1,  
        c='blue',  
        alpha=0.5,  
        label="Estimated Object"
    )

    camera_plotted = False
    for image in reconstruction.images.values():
        cam_fromworld=image.cam_from_world()
        cam_pos=cam_fromworld.translation
        if not camera_plotted:
            ax.scatter(cam_pos[0], cam_pos[1], cam_pos[2], c='red', marker='^', label="Camera")
            camera_plotted = True
        else:
            ax.scatter(cam_pos[0], cam_pos[1], cam_pos[2], c='red', marker='^')

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')

    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', 
             fontsize='x-large',
              markerscale=2.5,
              prop={'size': 20} )

    #plt.title("Estimated Object + Camera")
    plt.tight_layout()  
    plt.show()

In [81]:
def vis_interativa_camera_objeto(reconstruction):

    points_3d = []
    
    for point3D_id, point3D in reconstruction.points3D.items():
        points_3d.append(point3D.xyz)  

    points_3d = np.array(points_3d)
    
    cam_x_coords, cam_y_coords, cam_z_coords = [], [], []

    for image in reconstruction.images.values():
        cam_fromworld = image.cam_from_world()
        cam_pos = cam_fromworld.translation

        cam_x_coords.append(cam_pos[0])
        cam_y_coords.append(cam_pos[1])
        cam_z_coords.append(cam_pos[2])

    fig = go.Figure()
    fig.add_trace(
        go.Scatter3d(
            x=points_3d[:, 0],
            y=points_3d[:, 1],
            z=points_3d[:, 2],
            mode='markers',
            marker=dict(size=1.5, color='blue', opacity=0.6),
            name='Estimated Object'))

    fig.add_trace(
        go.Cone(
            x=cam_x_coords, y=cam_y_coords, z=cam_z_coords,
            u=-np.array(cam_x_coords), v=-np.array(cam_y_coords), w=-np.array(cam_z_coords),
            sizemode="absolute", sizeref=20, anchor="tip",
            colorscale='Reds', showscale=False, name='Camera',
            showlegend=True))

    fig.update_layout(
        title_text="Visualização da Nuvem Esparsa 3D",
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            aspectratio=dict(x=1, y=1, z=1)
        ),
        margin=dict(l=0, r=0, b=0, t=40),
        showlegend=True,
        legend=dict(
            x=0.8,
            y=0.9,
            xanchor='center',
            yanchor='top',
            bgcolor='rgba(255, 255, 255, 0.7)',
            bordercolor='rgba(0, 0, 0, 0.5)',
            borderwidth=1
        )
    )

    fig.show()

In [82]:
def gerar_malha(reconstruction, alpha = 1, n_interactions_smooth = 15):

    points = np.array([p.xyz for p in reconstruction.points3D.values()])

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
   
    print(f"Pontos originais: {len(pcd.points)}")
    print("Limpando outliers da nuvem de pontos...")
    pcd, ind = pcd.remove_statistical_outlier(nb_neighbors=20,
                                                    std_ratio=2.0)

    print(f"Pontos após limpeza: {len(pcd.points)}")
    mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha=alpha)
    
    mesh = mesh.remove_degenerate_triangles()
    mesh = mesh.remove_duplicated_triangles()
    mesh = mesh.remove_duplicated_vertices()
    mesh = mesh.remove_non_manifold_edges()


    mesh = mesh.filter_smooth_laplacian(number_of_iterations=n_interactions_smooth)
    mesh.compute_vertex_normals()
    return mesh


In [ ]:

CAMERA = True
METRICAS = True
VIS_ESTATICA = True
VIS_INTERATIVA = True
MALHA_3D = True

# Melhores valores de alpha para visualização da malha encontrados de forma empírica
alphas_imagens = {
1: 0.4,
2: 0.7,
3: 0.4,
4: 1,
}

for conjunto_imagens in [1,2,3,4]:
    project_path = Path(f'colmap_project_{conjunto_imagens}')
    database_path = project_path / 'database.db'
    output_path = project_path / 'sparse/0'
    
    reconstruction = pycolmap.Reconstruction(output_path) 
    
    print(f'=============={conjunto_imagens}==============')

    if CAMERA:
        plot_cameras(reconstruction)
    
    if METRICAS:
        reprojection_errors = np.array([point3D.error for point3D in reconstruction.points3D.values()])
        mean_reprojection_error = np.mean(reprojection_errors)
        median_reprojection_error = np.median(reprojection_errors)
        num_sparse_points = len(reconstruction.points3D)
        
        print(f"Número de Pontos Esparsos: {num_sparse_points}")
        print(f"Erro de Reprojeção Médio: {mean_reprojection_error:.4f} pixels")
        print(f"Erro de Reprojeção Mediano: {median_reprojection_error:.4f} pixels \n")
    
    if VIS_ESTATICA:
        vis_estatica_camera_objeto(reconstruction)

    if VIS_INTERATIVA:
        vis_interativa_camera_objeto(reconstruction)

    if MALHA_3D:
        output_malha3D_path = f'{project_path}/{conjunto_imagens}_malha3D.ply'
        mesh = gerar_malha(reconstruction, alpha=alphas_imagens[conjunto_imagens])
        print(f"Salvando a malha em {output_malha3D_path}")
        o3d.io.write_triangle_mesh(output_malha3D_path, mesh)
        print("Malha salva com sucesso!")
        #print("Visualizando a malha...")
        #o3d.visualization.draw_geometries([mesh])

colmap_project_1
colmap_project_1\database.db
colmap_project_1\sparse\0
colmap_project_1\1_malha3D.ply


ValueError: [reconstruction.cc:736] rigs, cameras, frames, images, points3D files do not exist at colmap_project_1\sparse\0